In [62]:
import os
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [63]:
def get_df(path):
    df = pd.read_csv(path, index_col='Unnamed: 0')
    return df

def get_val_loss(df):
    return df.val_loss.values[-1]

def get_val_nll(df):
    return df.val_nll.values[-1]

def get_last_epoch(df):
    return df.iloc[-1]

In [77]:
def get_nonlearned_hyperparameters(experiments_path, lr_0s, ns, random_states, weight_decays):
    columns = ['lr_0', 'n', 'random_state', 'weight_decay']
    df = pd.DataFrame(columns=columns)
    for n, random_state in itertools.product(ns, random_states):
        best_val_loss = np.inf
        best_hyperparameters = None
        for lr_0, weight_decay in itertools.product(lr_0s, weight_decays):
            model_name = 'nonlearned_lr_0={}_n={}_random_state={}_weight_decay={}'\
            .format(lr_0, n, random_state, weight_decay)
            path =  '{}/{}.csv'.format(experiments_path, model_name)
            val_loss = get_val_loss(get_df(path))
            if val_loss < best_val_loss: best_val_loss = val_loss; best_hyperparameters = [lr_0, n, random_state, weight_decay]
        df.loc[df.shape[0]] = best_hyperparameters
    return df
    
def get_learned_hyperparameters(experiments_path, lr_0s, ns, prior_scales, random_states, weight_decays):
    columns = ['lr_0', 'n', 'prior_scale', 'random_state', 'weight_decay']
    df = pd.DataFrame(columns=columns)
    for n, random_state in itertools.product(ns, random_states):
        best_val_nll = np.inf
        best_hyperparameters = None
        for prior_scale in prior_scales:
            best_val_loss = np.inf
            best_model_name = None
            best_lr_0 = None
            best_weight_decay = None
            for lr_0, weight_decay in itertools.product(lr_0s, weight_decays):
                model_name = 'learned_lr_0={}_n={}_prior_scale={}_random_state={}_weight_decay={}'\
                .format(lr_0, n, prior_scale, random_state, weight_decay)
                path =  '{}/{}.csv'.format(experiments_path, model_name)
                val_loss = get_val_loss(get_df(path))
                if val_loss < best_val_loss: best_val_loss = val_loss; best_model_name = model_name; best_lr_0 = lr_0; best_weight_decay = weight_decay
            path = '{}/{}.csv'.format(experiments_path, best_model_name)
            val_nll = get_val_nll(get_df(path))
            if val_nll < best_val_nll: best_val_nll = val_nll; best_hyperparameters = [best_lr_0, n, prior_scale, random_state, best_weight_decay]
        df.loc[df.shape[0]] = best_hyperparameters
    return df

def get_nonlearned_results(df, experiments_path):
    columns = ['n', 'random_state', 'method', 'test_auroc', 'test_loss', 'test_nll', 
               'test_prior', 'train_auroc', 'train_loss', 'train_nll', 'train_prior', 
               'val_auroc', 'val_loss', 'val_nll', 'val_prior']
    results = pd.DataFrame(columns=columns)
    for row_index, row in df.iterrows():
        model_name = 'nonlearned_lr_0={}_n={}_random_state={}_weight_decay={}'\
        .format(row.lr_0, int(row.n), int(row.random_state), row.weight_decay)
        path =  '{}/{}.csv'.format(experiments_path, model_name)
        last_epoch = get_last_epoch(get_df(path))
        results_row = [int(row.n), int(row.random_state), 'nonlearned', 
                       last_epoch.test_auroc, last_epoch.test_loss, 
                       last_epoch.test_nll, last_epoch.test_prior, 
                       last_epoch.train_auroc, last_epoch.train_loss, 
                       last_epoch.train_nll, last_epoch.train_prior, 
                       last_epoch.val_auroc, last_epoch.val_loss, 
                       last_epoch.val_nll, last_epoch.val_prior]
        results.loc[results.shape[0]] = results_row
    return results

def get_learned_results(df, experiments_path):
    columns = ['n', 'random_state', 'method', 'test_auroc', 'test_loss', 'test_nll', 
               'test_prior', 'train_auroc', 'train_loss', 'train_nll', 'train_prior', 
               'val_auroc', 'val_loss', 'val_nll', 'val_prior']
    results = pd.DataFrame(columns=columns)
    for row_index, row in df.iterrows():
        model_name = 'learned_lr_0={}_n={}_prior_scale={}_random_state={}_weight_decay={}'\
        .format(row.lr_0, int(row.n), row.prior_scale, int(row.random_state), row.weight_decay)
        path =  '{}/{}.csv'.format(experiments_path, model_name)
        last_epoch = get_last_epoch(get_df(path))
        results_row = [int(row.n), int(row.random_state), 'learned', 
                       last_epoch.test_auroc, last_epoch.test_loss, 
                       last_epoch.test_nll, last_epoch.test_prior, 
                       last_epoch.train_auroc, last_epoch.train_loss, 
                       last_epoch.train_nll, last_epoch.train_prior, 
                       last_epoch.val_auroc, last_epoch.val_loss, 
                       last_epoch.val_nll, last_epoch.val_prior]
        results.loc[results.shape[0]] = results_row
    return results

In [92]:
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/HAM10000/8:1:1'
lr_0s = np.logspace(-1, -4, num=4)
ns = [1000]
random_states = [1001, 2001, 3001]
#random_states = [4001, 5001, 6001]
weight_decays = np.append(np.logspace(-2, -6, num=5), 0)
nonlearned_hyperparameters = get_nonlearned_hyperparameters(experiments_path, lr_0s, ns, random_states, weight_decays)
nonlearned_hyperparameters

,lr_0,n,random_state,weight_decay
0,0.0001,1000.0,1001.0,0.01
1,0.0100,1000.0,2001.0,0.00
2,0.0001,1000.0,3001.0,0.01


In [94]:
lr_0s = np.logspace(-1, -4, num=4)
ns = [1000]
prior_scales = np.logspace(0, 9, num=10)
random_states = [1001, 2001, 3001]
#random_states = [4001, 5001, 6001]
weight_decays = np.append(np.logspace(-2, -6, num=5), 0)
learned_hyperparameters = get_learned_hyperparameters(experiments_path, lr_0s, ns, prior_scales, random_states, weight_decays)
learned_hyperparameters

,lr_0,n,prior_scale,random_state,weight_decay
0,0.0100,1000.0,1.000000e+09,1001.0,0.010
1,0.0100,1000.0,1.000000e+09,2001.0,0.001
2,0.0001,1000.0,1.000000e+06,3001.0,0.010


In [95]:
nonlearned_results = get_nonlearned_results(nonlearned_hyperparameters, experiments_path)
learned_results = get_learned_results(learned_hyperparameters, experiments_path)
results = pd.concat([nonlearned_results, learned_results])
grouped_results = results.groupby(['n', 'method']).agg(lambda x: tuple(x))
columns = ['test_auroc', 'train_auroc', 'val_auroc']
for column in columns:
    grouped_results['{}_mean'.format(column)] = grouped_results[column].apply(lambda item: np.mean(item))
    grouped_results['{}_std'.format(column)] = grouped_results[column].apply(lambda item: np.std(item))
grouped_results = grouped_results.reset_index()

In [96]:
grouped_results[['random_state', 'method', 'train_auroc_mean', 'train_auroc_std', 'val_auroc_mean', 
                 'val_auroc_std', 'test_auroc_mean', 'test_auroc_std']]

,random_state,method,train_auroc_mean,train_auroc_std,val_auroc_mean,val_auroc_std,test_auroc_mean,test_auroc_std
0,"(1001, 2001, 3001)",learned,1.0,0.0,0.862205,0.049993,0.834889,0.034321
1,"(1001, 2001, 3001)",nonlearned,1.0,0.0,0.878012,0.045251,0.856937,0.023444
